# Classification
we will predict the probability that a specific data point or entry is in all the classes it can be

we will use different properties of flower to predict what species of flower it is

In [1]:
import tensorflow as tf 
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd 
import numpy as np 

## Dataset
ths dataset seperates flowers into 3 classes of species

* setosa
* versicolor
* virginica

the information about each flower is provided as the following:
* sepal length
* sepal width
* petal length
* petal width

In [2]:
CSV_COLUMN_NAMES = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth", "Species"] # Pandas needs the headers to make a dataframe
SPECIES = ['Setosa', 'Versicolor', 'Virginica'] # species names for classification

In [17]:
# the data files are stored in a python module within tensorflow called keras
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

In [18]:
train.head() # in centimeters

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [19]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.shape

(120, 4)

In [20]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

## Input function
there are no epochs and the batch size is larger. We dont create a function in the input function
we must create the tensor dataset to be used by tensor flow

if the model is training, it repeats and shuffles 

In [21]:
def input_fn(features, labels, training=True, batch_size=256):
    # convert inputs into a tensorflow dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # shuffle and repeat if in training mode
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)


In [24]:
# feature columns, loop through all the keys in the training set and append to feature columns

my_feature_column = []
for key in train.keys(): # get the headers from the CSV file train
    my_feature_column.append(tf.feature_column.numeric_column(key=key)) # append the key as a feature column into the tensor
print(my_feature_column)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


## Model
we will be using a classification model. We have two main types of model we can use for our purposes. 

`DNNClassifier` (deep neural network)

`LinearClassifier` (operates much like linear regression, we get the probability of being a type rather than a precise value)

DNN is the best in this case because we are not certain of a linear relationship in the data
so lets go!

In [25]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_column, 
    # Two hidden layers of 30 and 10 nodes respectively
    hidden_units=[30, 10], 
    # the model must choose between 3 classes
    n_classes=3
    )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ahuma\\AppData\\Local\\Temp\\tmpnj94zelq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [51]:
# we would like to use a lambda instead of using nested functions for the input function.
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=6000 # go through the dataset until 5000 datapoints have been processed
)

# steps per second is displayed. The lower the loss the better. Current step also displayed

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ahuma\AppData\Local\Temp\tmpnj94zelq\model.ckpt-25000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 25000...
INFO:tensorflow:Saving checkpoints for 25000 into C:\Users\ahuma\AppData\Local\Temp\tmpnj94zelq\model.ckpt.
INFO:tensorflow:C:\Users\ahuma\AppData\Local\Temp\tmpnj94zelq\model.ckpt-25000.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:C:\Users\ahuma\AppData\Local\Temp\tmpnj94zelq\model.ckpt-25000.index
INFO:tensorflow:0
INFO:tensorflow:C:\Users\ahuma\AppData\Local\Temp\tmpnj94zelq\model.ckpt-25000.meta
INFO:tensorflow:100
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 25000...
INFO:tensorflow:loss = 0.09397784, step = 25000
INFO:tensorflow:globa

In [52]:
# evaluate the model
eval_result = classifier.evaluate(input_fn=lambda:input_fn(test, test_y, training=False))
print(f"\nTest set accuracy: {eval_result['accuracy']}\n")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-07-12T14:10:08
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ahuma\AppData\Local\Temp\tmpnj94zelq\model.ckpt-31000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.36600s
INFO:tensorflow:Finished evaluation at 2022-07-12-14:10:08
INFO:tensorflow:Saving dict for global step 31000: accuracy = 1.0, average_loss = 0.08030086, global_step = 31000, loss = 0.08030086
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 31000: C:\Users\ahuma\AppData\Local\Temp\tmpnj94zelq\model.ckpt-31000

Test set accuracy: 1.0



In [53]:
print(eval_result)
#print(test_y.loc[2])
#print(eval_result[2]) # a list of all the different dictionary that represent each element in the eval dataset

{'accuracy': 1.0, 'average_loss': 0.08030086, 'loss': 0.08030086, 'global_step': 31000}


100% accuracy!

## making predictions
we will make a script to take user input and predict what kind of flower it will be

In [63]:
def input_function(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
    # create an input with features, with no lables so we can get an answer

features = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]
predict = {} # Store the values for whcih we want to predict the class

print("Please type numeric values as prompted.")
for feature in features: # for each feature we wait for a valid response, and we add that to our dictionary
    valid = True
    val = input(f"{feature}: ") # take user input of feature value
    predict[feature] = [float(val)] #values to predict must be in a lsit for tensorflow. We assemble the keys and values in the dictionary. 
    # {predict[feature]:[float(val)]}

predictions = classifier.predict(input_fn=lambda: input_function(predict)) # every prediction comes as dictionary
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id] # get accuracy of prediction

    print(f'Prediction is {SPECIES[class_id]}, {100 * probability}%')
    print(pred_dict)

Please type numeric values as prompted.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ahuma\AppData\Local\Temp\tmpnj94zelq\model.ckpt-31000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is Versicolor, 60.763633251190186%
{'logits': array([-0.499649 ,  1.0321745,  0.187227 ], dtype=float32), 'probabilities': array([0.13133524, 0.60763633, 0.26102838], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1, 2]), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
